## Step 1: Import Libraries

In [1]:
from datetime import datetime
import polars as pl
import pandas as pd
import pyarrow as pa
from typing import List
import duckdb
import yfinance as yf

## Step 2: Import Symbols

In [2]:
def load_symbols(file_path: str) -> List[str]:
    """Load symbols from a text file"""
    try:
        with open(file_path, 'r') as f:
            symbols = [line.strip() for line in f if line.strip()]
        print(f"Loaded {len(symbols)} symbols from {file_path}")
        return symbols
    except Exception as e:
        print(f"Error loading symbols: {str(e)}")
        return []

symbols_file = '../tickers.txt'
symbols = load_symbols(symbols_file)

if symbols:
    print("Symbols:", symbols)
else:
    print("No symbols loaded.")

Loaded 60 symbols from ../tickers.txt
Symbols: ['AAPL', 'MSFT', 'GOOGL', 'GOOG', 'META', 'NVDA', 'AVGO', 'ORCL', 'CRM', 'ACN', 'ADBE', 'CSCO', 'INTC', 'NFLX', 'DIS', 'CMCSA', 'VZ', 'T', 'AMZN', 'TSLA', 'HD', 'MCD', 'NKE', 'SBUX', 'TGT', 'LOW', 'WMT', 'PG', 'KO', 'PEP', 'COST', 'BRK-B', 'JPM', 'BAC', 'WFC', 'GS', 'MS', 'BLK', 'UNH', 'JNJ', 'PFE', 'ABBV', 'MRK', 'LLY', 'CAT', 'BA', 'HON', 'UPS', 'RTX', 'GE', 'XOM', 'CVX', 'COP', 'SLB', 'LIN', 'APD', 'ECL', 'PLD', 'AMT', 'CCI']


## Step 3: Extract Data from Yfinance into Pandas

In [3]:
start_date = '2020-01-01'
end_date = datetime.today().strftime('%Y-%m-%d')

# Download prices from yfinance
prices = yf.download(symbols, start=start_date, end=end_date, group_by='ticker')

# Check if the data has a MultiIndex (due to multiple symbols)
if isinstance(prices.columns, pd.MultiIndex):
    # Flatten the MultiIndex columns
    prices.columns = ['_'.join(filter(None, map(str, col))) for col in prices.columns]

# Reset index and melt the DataFrame to include a "symbol" column
prices = prices.copy()  # Avoid fragmentation issues
prices.reset_index(inplace=True)
prices = prices.melt(id_vars=["Date"], var_name="Metric", value_name="Value")
prices[["Symbol", "Metric"]] = prices["Metric"].str.extract(r'([^_]+)_(.+)')
prices = prices.pivot(index=["Date", "Symbol"], columns="Metric", values="Value").reset_index()

display(prices)

[                       0%                       ]

[*                      3%                       ]  2 of 60 completed

[**                     5%                       ]  3 of 60 completed

[***                    7%                       ]  4 of 60 completed

[****                   8%                       ]  5 of 60 completed

...

[**********************95%*********************  ]  57 of 60 completed

[**********************97%********************** ]  58 of 60 completed

[**********************98%********************** ]  59 of 60 completed

[*********************100%***********************]  60 of 60 completed

Metric,Date,Symbol,Close,High,Low,Open,Volume
0,2020-01-02,AAPL,72.796013,72.856606,71.545380,71.799866,135480400.0
1,2020-01-02,ABBV,72.264893,72.281029,71.425635,71.885612,5639200.0
2,2020-01-02,ACN,195.263565,196.908187,194.018490,195.923277,2431100.0
3,2020-01-02,ADBE,334.429993,334.480011,329.170013,330.000000,1990100.0
4,2020-01-02,AMT,200.433929,202.758431,200.004110,201.468987,1426000.0
...,...,...,...,...,...,...,...
75655,2025-01-06,UPS,124.209999,125.000000,123.809998,124.699997,5049200.0
75656,2025-01-06,VZ,39.610001,40.330002,39.580002,40.290001,27094900.0
75657,2025-01-06,WFC,72.029999,73.250000,71.529999,71.930000,15565100.0
75658,2025-01-06,WMT,91.430000,92.470001,90.669998,90.830002,14515300.0


## Step 4: Convert Pandas to Polars

In [4]:
# Convert to Polars DataFrame
df = pl.from_pandas(prices)

# View the Polars DataFrame
print(df)

shape: (75_660, 7)
┌─────────────────────┬────────┬────────────┬────────────┬────────────┬────────────┬────────────┐
│ Date                ┆ Symbol ┆ Close      ┆ High       ┆ Low        ┆ Open       ┆ Volume     │
│ ---                 ┆ ---    ┆ ---        ┆ ---        ┆ ---        ┆ ---        ┆ ---        │
│ datetime[ns]        ┆ str    ┆ f64        ┆ f64        ┆ f64        ┆ f64        ┆ f64        │
╞═════════════════════╪════════╪════════════╪════════════╪════════════╪════════════╪════════════╡
│ 2020-01-02 00:00:00 ┆ AAPL   ┆ 72.796013  ┆ 72.856606  ┆ 71.54538   ┆ 71.799866  ┆ 1.354804e8 │
│ 2020-01-02 00:00:00 ┆ ABBV   ┆ 72.264893  ┆ 72.281029  ┆ 71.425635  ┆ 71.885612  ┆ 5.6392e6   │
│ 2020-01-02 00:00:00 ┆ ACN    ┆ 195.263565 ┆ 196.908187 ┆ 194.01849  ┆ 195.923277 ┆ 2.4311e6   │
│ 2020-01-02 00:00:00 ┆ ADBE   ┆ 334.429993 ┆ 334.480011 ┆ 329.170013 ┆ 330.0      ┆ 1.9901e6   │
│ 2020-01-02 00:00:00 ┆ AMT    ┆ 200.433929 ┆ 202.758431 ┆ 200.00411  ┆ 201.468987 ┆ 1.426e6    │
│

## Step 5: Write Polars to Parquet

In [5]:
output_dir = "../../../data/finance"

# Write DataFrame to Parquet
df.write_parquet(f'{output_dir}/historical_stock_quotes_{start_date}_to_{end_date}.parquet')

## Step 6: Read Parquet (Validate)

In [6]:
pl.scan_parquet(f'{output_dir}/historical_stock_quotes_{start_date}_to_{end_date}.parquet').head().collect()

Date,Symbol,Close,High,Low,Open,Volume
datetime[ns],str,f64,f64,f64,f64,f64
2020-01-02 00:00:00,"""AAPL""",72.796013,72.856606,71.54538,71.799866,1.354804e8
2020-01-02 00:00:00,"""ABBV""",72.264893,72.281029,71.425635,71.885612,5.6392e6
2020-01-02 00:00:00,"""ACN""",195.263565,196.908187,194.01849,195.923277,2.4311e6
2020-01-02 00:00:00,"""ADBE""",334.429993,334.480011,329.170013,330.0,1.9901e6
2020-01-02 00:00:00,"""AMT""",200.433929,202.758431,200.00411,201.468987,1.426e6
